# 1. 문서의 내용을 읽음.

In [1]:
# %pip install python-docx

In [ ]:
from docx import Document, document

document = Document('./tax.docx')
full_text = ''
for index, paragraph in enumerate(document.paragraphs):
    full_text += f'{paragraph.text}\n'

print(full_text)

# 2. 문서를 쪼갠다.

In [3]:
# %pip install tiktoken

In [4]:
import tiktoken

def split_text(full_text, chunk_size):
    encoder = tiktoken.encoding_for_model("gpt-4o")
    total_encoding = encoder.encode(full_text)
    total_token_count = len(total_encoding)
    text_list = []
    for i in range(0, total_token_count, chunk_size):
        chunk = total_encoding[i: i+chunk_size]
        decoded = encoder.decode(chunk)
        text_list.append(decoded)
    
    return text_list

In [5]:
# 에러 (chunk_list의 length가 100개 이상인 경우)
# chunk_list = split_text(full_text, 1500)

# 작동 (chunk_list의 length가 100개 미만인 경우)
chunk_list = split_text(full_text, 1700)

In [6]:
len(chunk_list)

95

# 3. 문서 임베딩

In [ ]:
# %pip install chromadb

In [8]:
import chromadb

chroma_client = chromadb.Client()

In [9]:
collection_name = 'tax_collection'

In [10]:
import os

from chromadb.utils.embedding_functions import OpenAIEmbeddingFunction

upstage_api_key = os.getenv('UPSTAGE_API_KEY')

upstage_embedding = OpenAIEmbeddingFunction(
                api_key=upstage_api_key,
                # Upstage의 OpenAI 호환 엔드포인트
                api_base="https://api.upstage.ai/v1", 
                # Upstage의 임베딩 모델명
                model_name="solar-embedding-1-large-query"
            )

In [11]:
tax_collection = chroma_client.get_or_create_collection(collection_name, embedding_function=upstage_embedding)

In [12]:
id_list = []
for index in range(len(chunk_list)):
    id_list.append(f'{index}')

In [13]:
len(chunk_list)

95

In [14]:
len(id_list)

95

In [15]:
tax_collection.add(documents=chunk_list, ids=id_list)

# 4. 유사도 검색

In [16]:
query = '연봉 5천만원인 직장인의 소득세는 얼마인가요?'
retrived_doc = tax_collection.query(query_texts=query, n_results=3)

In [ ]:
retrived_doc['documents'][0]

# 5. LLM 질의

In [ ]:
# %pip install openai

In [19]:
from openai import OpenAI
client=OpenAI(
    api_key=upstage_api_key,
    base_url="https://api.upstage.ai/v1"
)

response = client.chat.completions.create(
    model="solar-pro2",
    messages=[
        {"role": "system", "content":f"당신은 한국의 소득세 전문가 입니다. 아래 내용을 참고해서 사용자의 질문에 답변해주세요. {retrived_doc['documents'][0]}"},
        {"role": "user", "content": query}
    ]
)

In [20]:
response.choices[0].message.content

'연봉 5,000만원인 직장인의 소득세를 계산할 때는 **근로소득공제, 종합소득과세표준, 세율, 세액공제** 등을 단계별로 고려해야 합니다. 아래는 간단한 계산 절차입니다:\n\n---\n\n### 1. **근로소득공제 적용 (총급여액 기준)**\n- **총급여액**: 5,000만원  \n- **근로소득공제율** (2024년 기준):  \n  - 5,500만원 이하: 총급여액 × 70% + 110만원  \n  - 계산:  \n    \\[\n    5,000만원 \\times 70\\% + 110만원 = 3,610만원\n    \\]\n- **공제 후 소득금액**:  \n  \\[\n  5,000만원 - 3,610만원 = 1,390만원\n  \\]\n\n---\n\n### 2. **종합소득과세표준 계산**\n- **기본공제 등 추가 공제** (예시로 본인만 있는 경우):  \n  - 기본공제 (1인당 150만원) = 150만원  \n  - **과세표준**:  \n    \\[\n    1,390만원 - 150만원 = 1,240만원\n    \\]\n  - (※ 실제로는 부양가족, 연금보험료, 의료비 등 추가 공제 가능)\n\n---\n\n### 3. **산출세액 계산 (2024년 종합소득세율)**\n- **세율 적용**:  \n  - 1,200만원 이하: 6%  \n  - 1,200만원 초과 ~ 4,600만원 이하: 15% (1,200만원 초과분에 적용)  \n  - 계산:  \n    \\[\n    (1,200만원 \\times 6\\%) + (40만원 \\times 15\\%) = 72만원 + 6만원 = 78만원\n    \\]\n\n---\n\n### 4. **세액공제 적용**\n- **근로소득세액공제** (제59조):  \n  - 총급여액 3,300만원 초과 7,000만원 이하 → **74만원 - (총급여액 - 3,300만원) × 8/1,000**  \n    \\[\n    74만원 - (5,000만원 - 3,300만원) \\times 0.008 =